In [1]:
%load_ext sql

In [2]:
%sql postgresql://appdev@data:5432/appdev

'Connected: appdev@appdev'

Using a single join and a grouping, write a query that allows you to find the forename, surname, driverid and the total number of wins for each driver in the f1db schema.

In [5]:
%sql SELECT * FROM drivers limit 1;

1 rows affected.


driverid,driverref,number,code,forename,surname,dob,nationality,url
1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton


In [10]:
%sql SELECT * FROM results limit 1;

1 rows affected.


resultid,raceid,driverid,constructorid,number,grid,position,positiontext,positionorder,points,laps,time,milliseconds,fastestlap,rank,fastestlaptime,fastestlapspeed,statusid
1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1


In [11]:
%sql SELECT * FROM circuits limit 1;

1 rows affected.


circuitid,circuitref,name,location,country,lat,lng,alt,url,position
1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit,"(144.968,-37.8497)"


In [12]:
%sql SELECT * FROM races limit 1;

1 rows affected.


raceid,year,round,circuitid,name,date,time,url
1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_Grand_Prix


In [31]:
%sql SELECT forename, surname, driverid, count(position) FROM results INNER JOIN drivers USING (driverid) WHERE position = 1 GROUP BY forename, surname, driverid ORDER BY count(position) DESC limit 10;

10 rows affected.


forename,surname,driverid,count
Michael,Schumacher,30,91
Lewis,Hamilton,1,60
Alain,Prost,117,51
Sebastian,Vettel,20,46
Ayrton,Senna,102,41
Fernando,Alonso,4,32
Nigel,Mansell,95,31
Jackie,Stewart,328,27
Niki,Lauda,182,25
Jim,Clark,373,25


Write a query that joins the tables results, constructors and drivers to show the amount of times a driver has driven a car from a constructor. In other words how many times one driver (for instance Schumacher) has driven a constructor (for instance Mercedes). Your table should have three fields: drivers.driverref, constructors.name and count(*) (showing the number of times the driver has driven the constructor).

In [21]:
%sql SELECT * FROM constructors limit 1;

1 rows affected.


constructorid,constructorref,name,nationality,url
1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren


In [33]:
%sql SELECT drivers.driverref, constructors.name, count(*) FROM results INNER JOIN drivers USING (driverid) INNER JOIN constructors USING (constructorid) GROUP BY drivers.driverref, constructors.name ORDER BY count(*) DESC limit 10;

10 rows affected.


driverref,name,count
michael_schumacher,Ferrari,181
coulthard,McLaren,150
massa,Ferrari,140
button,McLaren,137
rosberg,Mercedes,136
hakkinen,McLaren,133
laffite,Ligier,132
webber,Red Bull,129
raikkonen,Ferrari,126
vettel,Red Bull,113


Now we have found the most driven vehicles even for those who didn't finish the race. Extend the query from 2 by removing all tuples from the result where the status is not "Finished".

In [36]:
%sql SELECT * FROM status limit 1;

1 rows affected.


statusid,status
1,Finished


In [34]:
%sql SELECT drivers.driverref, constructors.name, count(*) FROM results INNER JOIN drivers USING (driverid) INNER JOIN constructors USING (constructorid) INNER JOIN status USING (statusid) WHERE status = 'Finished' GROUP BY drivers.driverref, constructors.name ORDER BY count(*) DESC limit 10;

10 rows affected.


driverref,name,count
michael_schumacher,Ferrari,141
massa,Ferrari,112
rosberg,Mercedes,109
webber,Red Bull,100
vettel,Red Bull,97
raikkonen,Ferrari,92
alonso,Ferrari,87
button,McLaren,87
coulthard,McLaren,85
hamilton,Mercedes,85


Create a third (and new) query that finds the amount of milliseconds spent in pitstop (see the pitstops relation) by each unique combinations of driverid and raceid.

    Hint: use the sum aggregate function to find the total amount of milliseconds
    Hint: i recommend making this into a view


In [37]:
%sql SELECT * FROM pitstops limit 1;

1 rows affected.


raceid,driverid,stop,lap,time,duration,milliseconds
841,1,1,16,17:28:24,23.227,23227


In [41]:
%sql SELECT driverid, raceid, sum(milliseconds) FROM pitstops GROUP BY driverid, raceid ORDER BY sum(milliseconds) limit 10; 

10 rows affected.


driverid,raceid,sum
154,909,18928
17,877,19066
4,877,19368
18,877,19455
155,877,19529
13,877,19585
18,871,19668
822,955,19732
10,877,19884
3,955,20058


In [88]:
%sql CREATE VIEW ex_4 AS SELECT driverid, raceid, sum(milliseconds) FROM pitstops GROUP BY driverid, raceid ORDER BY sum(milliseconds) limit 10;

Done.


[]

Time to put everything together. Use the query for 4 as a join subquery in query 3 (see slides on "a query within a query") to find the total amount of pitstop time, for each result in query 3

In [94]:
%%sql SELECT drivers.driverref, constructors.name, count(*), ex_4.sum FROM results 
    INNER JOIN drivers USING (driverid) 
    INNER JOIN constructors USING (constructorid) 
    INNER JOIN status USING (statusid) 
    INNER JOIN ex_4 USING (driverid)
    WHERE status = 'Finished' 
    GROUP BY drivers.driverref, constructors.name, ex_4.sum
    ORDER BY count(*)
    limit 10;

10 rows affected.


driverref,name,count,sum
kobayashi,Caterham,1,19529
grosjean,Renault,1,18928
kobayashi,Toyota,2,19529
button,Benetton,2,19668
glock,Marussia,2,19884
button,Benetton,2,19455
button,Williams,4,19455
button,Williams,4,19668
button,Renault,6,19455
button,Renault,6,19668


As a last thing we want to find the driver with the least pitsop time. However we want to control for the amount of races (see explanation below), so we need to include one last column: the average pitstop time per race. Your table should be sorted based on the ascending values of that average pitstop time.

In [98]:
%%sql SELECT drivers.driverref, constructors.name, count(*), ex_4.sum, ex_4.sum/count(results.raceid) AS avg FROM results 
    INNER JOIN drivers USING (driverid) 
    INNER JOIN constructors USING (constructorid) 
    INNER JOIN status USING (statusid) 
    INNER JOIN ex_4 USING (driverid)
    WHERE status = 'Finished' 
    GROUP BY drivers.driverref, constructors.name, ex_4.sum
    ORDER BY avg ASC
    limit 1;

1 rows affected.


driverref,name,count,sum,avg
massa,Ferrari,112,19585,174.8660714285714286
